In [383]:
import spacy
from spacy import displacy
from collections import Counter
from spacy import tokenizer
import nltk

import pandas as pd
import numpy as np

from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint
nlp = spacy.load('en_core_web_sm')

##nltk.download('words')
##nltk.download('punkt')
##nltk.download('maxent_ne_chunker')
##nltk.download('averaged_perceptron_tagger')
##nltk.download('state_union')##

In [384]:
import io
import math
import requests
import PyPDF2
import spacy
import string
import re
import gensim
from sklearn import svm

from sklearn.feature_extraction import text
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from pyLDA_visual import topic_visual
from Grid_Search_LDA import lda_gridsearch
from LDAGensim import lda_gensim_model

In [385]:
def get_org(doc):
    for i in doc.ents:
        if i.label_ == "ORG":
            organization = i.text
            return organization

In [386]:
path = './dataset.xlsx'
ds = pd.read_excel(path)
ds.head()


,Title,News,ESG_RELEVANT,COMPANY_NAME,SUB_PILLARS,NATURE_OF_HARM,SCALE_OF_IMPACT,CONTROVERSY_ASSESSMENT,STATUS,IS_DIRECT,FLAG
0,NaN,Italy is attempting to weaken a pledge 10 Euro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,The Dutch government on Thursday said it inten...,NaN,NaN,NaN,4.0,2.0,NaN,1.0,0.0,NaN
2,NaN,A South African project to decommission and re...,NaN,NaN,NaN,3.0,2.0,NaN,2.0,1.0,NaN
3,NaN,"The Galapagos Islands, whose thousands of uniq...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,European authorities fined Google a record $5....,NaN,NaN,NaN,4.0,3.0,NaN,0.0,0.0,NaN


In [387]:
ds["News"] = ds["News"]
ds["News"].count()

6

In [388]:
for index, x in ds.iterrows():
    temp = nlp(x["News"])
    ds.loc[index, "COMPANY_NAME"] = get_org(temp)
    if get_org(temp) == None :
        ds = ds.drop(index)


In [389]:
ds.head()

,Title,News,ESG_RELEVANT,COMPANY_NAME,SUB_PILLARS,NATURE_OF_HARM,SCALE_OF_IMPACT,CONTROVERSY_ASSESSMENT,STATUS,IS_DIRECT,FLAG
2,NaN,A South African project to decommission and re...,NaN,the World Bank,NaN,3.0,2.0,NaN,2.0,1.0,NaN
4,NaN,European authorities fined Google a record $5....,NaN,Google,NaN,4.0,3.0,NaN,0.0,0.0,NaN
5,NaN,A South African project to decommission and re...,NaN,the World Bank,NaN,2.0,2.0,NaN,2.0,1.0,NaN


In [390]:
stop_words = text.ENGLISH_STOP_WORDS

In [391]:
bigram_tf_idf_vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(2,2), min_df=0, use_idf=True)
bigram_tf_idf = bigram_tf_idf_vectorizer.fit_transform(ds.News)

# Extract bi-grams names
words = bigram_tf_idf_vectorizer.get_feature_names_out()

# extract our top 10 ngrams
total_counts = np.zeros(len(words))
for t in bigram_tf_idf:
    total_counts += t.toarray()[0]

count_dict = (zip(words, total_counts))
count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:10]
words = [w[0] for w in count_dict]
counts = [w[1] for w in count_dict]
x_pos = np.arange(len(words))

# Plot top 10 ngrams
#plt.figure(figsize=(15, 5))
#plt.subplot(title='10 most common bi-grams')
#sns.barplot(x_pos, counts, palette='Blues_r')
#plt.xticks(x_pos, words, rotation=90)
#plt.xlabel('bi-grams')
#plt.ylabel('tfidf')
#plt.show()

In [392]:
# %% Modeling : NMF (Non Negative Matrix factorization)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

word_tf_idf_vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,1))
word_tf_idf = word_tf_idf_vectorizer.fit_transform(ds.News)
n_top_words = 20

# Fit the NMF model Frobenius norm
print("Fitting the NMF model (Frobenius norm) with tf-idf feature")
nmf = NMF(n_components=15, random_state=42,
          alpha=.3, l1_ratio=.5).fit(word_tf_idf)

#print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = word_tf_idf_vectorizer.get_feature_names_out()
print_top_words(nmf, tfidf_feature_names, n_top_words)

Fitting the NMF model (Frobenius norm) with tf-idf feature
Topic #0:
world company friday fired fined financing european eskom decommission coal wednesday billion batteries bank authorities alter african abusing google market
Topic #1:
world company friday fired fined financing european eskom decommission coal wednesday billion batteries bank authorities alter african abusing google market
Topic #2:
world company friday fired fined financing european eskom decommission coal wednesday billion batteries bank authorities alter african abusing google market
Topic #3:
world company friday fired fined financing european eskom decommission coal wednesday billion batteries bank authorities alter african abusing google market
Topic #4:
world company friday fired fined financing european eskom decommission coal wednesday billion batteries bank authorities alter african abusing google market
Topic #5:
world company friday fired fined financing european eskom decommission coal wednesday billion ba

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [393]:

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

word_tf_vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1,1))
word_tf = word_tf_vectorizer.fit_transform(ds.News)

# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=5,
                                      learning_decay=0.3,
                                      #max_iter=10,               # Max learning iterations
                                      #learning_method='online',
                                      random_state=42,          # Random state
                                      #batch_size=128,            # n docs in each learning iter
                                      #evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      #n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit(word_tf)

print(lda_model)
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(word_tf))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(word_tf))

# See model parameters
print(lda_model.get_params())

#print_top_words(lda_model, word_tf_vectorizer.get_feature_names(), 20)

LatentDirichletAllocation(learning_decay=0.3, n_components=5, random_state=42)
Log Likelihood:  -276.2855433840814
Perplexity:  65.76873653197678
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.3, 'learning_method': 'batch', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 5, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': 42, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [394]:
# %% Gensim LDA for coherence score
#lda_gensim_model(ds) #(stat_lem_list)
# No. of Topics = 9
# Human intution of topic names:
#TOPIC 1(G) : Ethical investment
#TOPIC 2(E): Sustainable finance
#TOPIC 3(S): Value employee
#TOPIC 4(G): Code of Conduct
#TOPIC 5(E): Climate change
#TOPIC 6(E): renewable energy
#TOPIC 7(G): Customer centric
#Topic 8(G): Strong governance
#Topic 9(S): Support community
#topic_names = [  'ethical investment',  'Sustainable finance',  'Value employee',  'Code of Conduct',  'Climate change','renewable energy','Customer centric','Strong governance','Support community']

topic_names = ['Environment',
               'Customers',
               'Human rights and community',
               'Labor rights and suppy chain',
               'Governance'
              ]

In [395]:
#%% Topic Distribution for each of the statements

transformed = lda_model.transform(word_tf)
# find principal topic from distribution...
a = [topic_names[np.argmax(distribution)] for distribution in transformed]
# ... with associated probability
b = [np.max(distribution) for distribution in transformed]

esg_prob = pd.DataFrame(zip(a,b,transformed), columns=['SUB_PILLARS', 'probability', 'probabilities'])
ds["SUB_PILLARS"] = a
ds.head()

,Title,News,ESG_RELEVANT,COMPANY_NAME,SUB_PILLARS,NATURE_OF_HARM,SCALE_OF_IMPACT,CONTROVERSY_ASSESSMENT,STATUS,IS_DIRECT,FLAG
2,NaN,A South African project to decommission and re...,NaN,the World Bank,Governance,3.0,2.0,NaN,2.0,1.0,NaN
4,NaN,European authorities fined Google a record $5....,NaN,Google,Customers,4.0,3.0,NaN,0.0,0.0,NaN
5,NaN,A South African project to decommission and re...,NaN,the World Bank,Governance,2.0,2.0,NaN,2.0,1.0,NaN


In [396]:
x_train, x_test, y_train, y_test = train_test_split(word_tf, ds.NATURE_OF_HARM, test_size=0.2, random_state=42)
model_svc = svm.LinearSVC(multi_class='crammer_singer')
model_svc.fit(x_train,y_train)

y_pred= model_svc.predict(x_test)
print(y_pred)

[2.]


In [398]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(word_tf, ds.SCALE_OF_IMPACT, test_size=0.2, random_state=42)
model_svc = svm.LinearSVC(multi_class='crammer_singer')
model_svc.fit(x_train2,y_train2)

y_pred2= model_svc.predict(x_test2)
print(y_pred2)

[2.]


In [401]:
x_train3, x_test3, y_train3, y_test3 = train_test_split(word_tf, ds.STATUS, test_size=0.2, random_state=42)
model_svc = svm.LinearSVC(multi_class='crammer_singer')
model_svc.fit(x_train3,y_train3)

y_pred3= model_svc.predict(x_test3)
print(y_pred3)

[2.]


In [403]:
x_train4, x_test4, y_train4, y_test4 = train_test_split(word_tf, ds.IS_DIRECT, test_size=0.2, random_state=42)
model_svc = svm.LinearSVC(multi_class='crammer_singer')
model_svc.fit(x_train4,y_train4)

y_pred4= model_svc.predict(x_test4)
print(y_pred4)

[1.]


In [409]:
ds["CONTROVERSY_ASSESSMENT"] = (ds["NATURE_OF_HARM"] + ds["SCALE_OF_IMPACT"])/2
ds["CONTROVERSY_ASSESSMENT"] = ds["CONTROVERSY_ASSESSMENT"].apply(lambda x: math.floor(x))
ds["FLAG"] = ds["CONTROVERSY_ASSESSMENT"] + ds["STATUS"] + ds["IS_DIRECT"]
ds.sort_values("COMPANY_NAME", ascending = False)

,Title,News,ESG_RELEVANT,COMPANY_NAME,SUB_PILLARS,NATURE_OF_HARM,SCALE_OF_IMPACT,CONTROVERSY_ASSESSMENT,STATUS,IS_DIRECT,FLAG
2,NaN,A South African project to decommission and re...,NaN,the World Bank,Governance,3.0,2.0,2,2.0,1.0,5.0
5,NaN,A South African project to decommission and re...,NaN,the World Bank,Governance,2.0,2.0,2,2.0,1.0,5.0
4,NaN,European authorities fined Google a record $5....,NaN,Google,Customers,4.0,3.0,3,0.0,0.0,3.0
